1 Implement the AI Game Strategy
Part 1 –(a). Install the Python Libraries required for Game Strategy
1. Install the python libraries - collections, random, math, functools,
cache = functools.lru cache(10**6)
2. Implement a Game Class Constructor using action, is terminal, result, utility functions
3. A game is similar to a problem, but it has a terminal test instead of a goal test, and a
utility for each terminal state.
4. Create a game subclass and implement actions, result, is terminal, and utility.
5. You will also need to set the initial attribute to the initial state; this can be done in the
constructor.
6. Implement a Player Game using the Game Class Constructor.

In [ ]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
cache = functools.lru_cache(10**6)

In [ ]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of
        a goal test, and a utility for each terminal state. To create a game,
            subclass this class and implement actions, result, is_terminal,
                and utility. You will also need to set the .initial attribute to the
                    initial state; this can be done in the constructor."""

    def actions(self, state): #Removed extra indent
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

question 2

Implement the Game Strategy Algorithms
1. Implement the MiniMax Search Algorithm


In [ ]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move #Removed extra indent

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

infinity = math.inf

In [ ]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    ""Search all the way to the leaves."""

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

question 3

3. Implement random player(game,state) and player(search algorithm)

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an height by width board, needing k in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3): #Fixed indentation, renamed init to __init__
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board): print(board)


    def k_in_row(board, player, square, k):
        """True if player has k pieces in a line through square."""
        def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy)-1>=k
                  for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))
        class Board(defaultdict):
            """A board has the player to move, a cached utility value,
            and a dict of {(x, y): player} entries, where player is 'X' or 'O'., defaultdict will give key value"""
            empty = '.'
            off = '#'

            def __init__(self, width=8, height=8, to_move=None, **kwds): #Fixed indentation, renamed init to __init__
                self.dict.update(width=width, height=height, to_move=to_move, **kwds)

            def new(self, changes: dict, **kwds) -> 'Board':
                "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
                board = Board(width=self.width, height=self.height, **kwds)
                board.update(self)
                board.update(changes)
                return board

            def missing(self, loc):
                x, y = loc
                if 0 <= x < self.width and 0 <= y < self.height:
                    return self.empty
                else:
                    return self.off

            def __hash__(self): #Renamed hash to __hash__
                return hash(tuple(sorted(self.items()))) + hash(self.to_move)

            def __repr__(self): #Renamed repr to __repr__
                def row(y): return ' '.join(self[x, y] for x in range(self.width))
                return '\n'.join(map(row, range(self.height))) +  '\n'
            def random_player(game, state): return random.choice(list(game.actions(state)))

            def player(search_algorithm):
                """A game player who uses the specified search algorithm"""
                return lambda game, state: search_algorithm(game, state)[1]

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an height by width board, needing k in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3): #Fixed typo here, removed extra indentation
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}

question 4


Part 4 – Evaluate the AI Game Strategy using TicTocToe

1. Implement Game Strategy using play game(TicTacToe(), dict(X=random player,
O=player(alphabeta search)), verbose=True).utility

2. Implement Game strategy using play game(TicTacToe(), dict(X=player(alphabeta search),O=player(minimax search)), verbose=True).utility


In [7]:
import random
import math

# Assuming the previous TicTacToe class and random_player, minimax_player, alphabeta_player functions are defined

class TicTacToe:
    def __init__(self): #Fixed indentation, changed init to __init__
        """Initialize the game with an empty 3x3 board and 'X' as the first player."""
        self.board = [' '] * 9  # A list of 9 spaces representing the Tic-Tac-Toe grid
        self.current_player = 'X'  # 'X' always goes first

    def actions(self, state):
        """Return the list of available actions (empty spaces) on the board."""
        return [i for i in range(len(state[0])) if state[0][i] == ' ']

    def result(self, state, action):
        """Return the new game state after performing the action."""
        new_board = state[0][:]
        new_board[action] = state[1]  # Update the board with the player's move
        next_player = 'O' if state[1] == 'X' else 'X'
        return (new_board, next_player)

    def is_terminal(self, state):
        """Check if the game has ended (win or draw)."""
        return self.check_winner(state) is not None or ' ' not in state[0]

    def utility(self, state, player):
        """Return 1 if 'X' wins, -1 if 'O' wins, 0 for a draw."""
        winner = self.check_winner(state)
        if winner == 'X':
            return 1
        elif winner == 'O':
            return -1
        else:
            return 0

    def check_winner(self, state):
        """Check for a winner and return 'X', 'O', or None."""
        board = state[0]
        win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                         (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                         (0, 4, 8), (2, 4, 6)]             # Diagonals
        for (i, j, k) in win_conditions:
            if board[i] == board[j] == board[k] and board[i] != ' ':
                return board[i]
        return None


    def random_player(game, state):
        """Choose a random move from available actions."""
        available_actions = game.actions(state)
        return random.choice(available_actions)


    def minimax_player(game, state):
        """Choose the best move using the Minimax search algorithm."""
        def minimax_search(state):
            player = state[1]  # 'X' or 'O'

            def max_value(state):
                if game.is_terminal(state):
                    return game.utility(state, player), None
                v, move = -math.inf, None
                for a in game.actions(state):
                    v2, _ = min_value(game.result(state, a))
                    if v2 > v:
                        v, move = v2, a
                return v, move

            def min_value(state):
                if game.is_terminal(state):
                    return game.utility(state, player), None
                v, move = math.inf, None
                for a in game.actions(state):
                    v2, _ = max_value(game.result(state, a))
                    if v2 < v:
                        v, move = v2, a
                return v, move

            return max_value(state) if player == 'X' else min_value(state)

        _, best_move = minimax_search(state)
        return best_move


    def alphabeta_player(game, state):
        """Choose the best move using Alpha-Beta pruning."""
        def alphabeta_search(state):
            player = state[1]

